In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from ollama import Client
from IPython.display import display_markdown

from tool import ToolKit, ToolCallProcessor
from functions import summation, multiplication

#### Connect to local Ollama_server

In [3]:
client = Client(host="http://localhost:11434")

model_name = "qwen2.5:7b"

# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

### Registry functions

In [4]:
# Instanciar o registro de ferramentas
toolkit = ToolKit(tools=[summation, multiplication])

## Create planning agent

In [5]:
tools = "\n\n".join(
    [json.dumps(schema, indent=4) for schema in toolkit.tools_schemas()]
)


REACT_PROMPT = (
    """
You are a function calling AI model. You operate breaking a task given by a user's question into steps: <thought>, <tool_calls>, <tool_response>.
Take special attention to the functions params dtypes.
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags, here are the available tools:
<tools>
%s
</tools>

The reasoning and thoughts should be enclosed within <thought></thought> XML tags.
<thought>
thought/reasoning
</thought>

If a function call is needed and the function is listed, return a JSON object containing the function name and its arguments, enclosed within <tool_calls></tool_calls> XML tags.
<tool_calls>
{"name": <function-name>, "arguments": <args-json-object>, "id": <monotonically-increasing-id>}
</tool_calls>

The function call response will be enclosed within <tool_response></tool_response> XML tags.
<tool_response>
tool response
</tool_response>

The final response to the user will be enclosed within <response></response> XML tags.
<response>
Response after reasoning and acting (ReAct)
</response>

Example session:

<question>Can you verify if it's friday and if yes, can you show a message \"IT'S FRIDAY\"?</question>
<thought>I need to check if today is friday, if today is friday i should show a message saying \"IT'S FRIDAY\".First i will verify if it's friday.</thought>
<tool_calls>{\"name\": \"is_friday\",\"arguments\": {\"date\": \"10/01/2024\"}, \"id\": 0}</tool_calls>

Then you will be feeded with the result of the function call:
<tool_response>True</tool_response>

Then you can go to the next step with depends on first:
<thought>It's friday, now i should show a message:</thought>
<tool_calls>{\"name\": \"show_message\",\"arguments\": {\"message\": \"IT'S FRIDAY\"}, \"id\": 1}</tool_calls>

Then you will be feeded with the result of the function call: 
<tool_response>{"success": True}</tool_response>

And if no more function calls are needed, you can respond with:
<response>Today is friday! I showed a message remenbering you of this.</response>
"""
    % tools
)

In [6]:
import re


def is_final_response(content, pattern=r"<response>(.*?)</response>"):
    """
    Check if the input string contains <response></response> tags.

    Args:
        input_string (str): The string to check for <response> tags.

    Returns:
        bool: True if the tags are found, False otherwise.
    """
    # Search for the pattern in the input string
    match = bool(re.search(pattern, content, re.DOTALL))

    # Return True if the tags are found
    return match

In [ ]:
# Step 1
chat_history = [
    {
        "role": "system",
        "content": REACT_PROMPT,
    }
]

chat_history.append(
    {
        "role": "user",
        "content": "<question>Can you multiple 25 by 12 and sum the result by 3 and multiply by 12 and then sum by 354?</question>",
    }
)

final_response = None

while True:
    planning_request = client.chat(model=model_name, messages=chat_history)

    planning_request_content = planning_request["message"]["content"]

    if is_final_response(planning_request_content):
        final_response = planning_request_content
        break

    processor = ToolCallProcessor(planning_request_content)

    chat_history.append({"role": "assistant", "content": planning_request_content})

    for call in processor.calls:
        try:
            result = toolkit.get_tool_by_name(call.name).run(call.arguments)

            chat_history.append(
                {"role": "tool", "content": f"<tool_response>{result}</tool_response>"}
            )
        except Exception as ee:
            print(ee)

In [ ]:
chat_history

[{'role': 'system',
  'content': '\nYou are a function calling AI model. You operate breaking a task given by a user\'s question into steps: <thought>, <tool_calls>, <tool_response>.\nTake special attention to the functions params dtypes.\nYou may call one or more functions to assist with the user query.\nYou are provided with function signatures within <tools></tools> XML tags, here are the available tools:\n<tools>\n{\n    "name": "summation",\n    "doc": "\\n    Add two integers.\\n\\n    Parameters\\n    ----------\\n    x : int\\n        The first integer to be added.\\n    y : int, optional\\n        The second integer to be added (default is 0).\\n\\n    Returns\\n    -------\\n    int\\n        The sum of `x` and `y`.\\n    ",\n    "parameters": {\n        "properties": {\n            "x": {\n                "type": "int"\n            },\n            "y": {\n                "type": "int"\n            }\n        }\n    }\n}\n\n{\n    "name": "multiplication",\n    "doc": "\\n   

In [ ]:
print(final_response)

The final result of the calculation is 3990. 

<response>The result of multiplying 25 by 12, summing the result by 3, multiplying that by 12 and then summing by 354 is 3990.</response>
